In [6]:
from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor

In [ ]:
# Note: Use larger num_shots in evaluate_circ() than the bit size you need.
# User is supposed to verify by themselves if the random bit num length 
# returned with get_random() meets their minimum length requirements. Number
# returned is going to be <num_shots passed in to evaluate_circ() due to noise
# in real hardware and uncertainty of real measurements. This is not true for
# simulate_circ() as results here are 'idealized' and the condition
# num_shots == len(get_random()) will always hold.

# To use, first call init_circ() to initialize the circuit with 3 entangled qubits. 
# If you choose to run in a simulation (aer-backend), simply call simulate_circ()
# and pass in the number of shots requested (corresponding to final length of the
# randomly generated sequence of bits). Otherwise, call load_provider() to load
# the available real-hardware backends from IBM-Q, which assumes assumes account 
# token has been already saved, and then evaluate_circ() (again, pass in the requested
# shot count). Once the simulation/request-to-hardware is ran, call get_random() to
# have the random bit sequence returned.

In [49]:
class QBitGen:
    def __init__(self):
        self.__random_gen = ""
        self.__qrn_circ = None
        self.__failed_results = False
        self.__provider = None

    # Setup the circuit 
    def init_circ(self):
        qcirc = QuantumCircuit(3, 3)

        # Create a state of 3 entangled qbits
        qcirc.h(2)
        qcirc.cx(2, 0)
        qcirc.barrier()

        qcirc.h(1)
        qcirc.cx(1, 0)
        qcirc.barrier()

        # Perform measurements in z-basis
        qcirc.measure(0, 0)
        qcirc.measure(1, 1)
        qcirc.measure(2, 2)

        self.__qrn_circ = qcirc
        
    # Run the circuit in simulator
    def simulate_circ(self, num_shots=1024):        
        if(self.__qrn_circ == None):
            print("Initialize the circuit first")
            return
        
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(self.__qrn_circ, shots = num_shots, memory=True)
        result = aer_sim.run(qobj).result().get_memory()

        # Verify XOR-rule for obtained results
        random_gen = ""
        for measurement in result:
            if int(measurement[0]) ^ int(measurement[1]) != int(measurement[2]):
                print("Bad result found: " + measurement)
                self.__failed_results = True
                return

            # Append the measurement
            random_gen += measurement[0]

        self.__failed_results = False
        self.__random_gen = random_gen
    
    # Load the IBMQ account
    def load_provider(self):
        try:
            IBMQ.load_account()
            self.__provider = IBMQ.get_provider(hub='ibm-q')
        except IBMQProviderError:
            print("Could not fetch providers")
    
    # Return the random, generated sequence, or "" if error
    def get_random(self)->str:
        if self.__random_gen == "" or self.__failed_results == True:
            print("Could not fetch random, try to generate it again")
            return ""
        return self.__random_gen
    
    # Run on the real hardware
    def evaluate_circ(self, num_shots=100):
        if self.__qrn_circ == None:
            print("Initialize the circuit first")
            return
        
        if self.__provider == None:
            print("Load the available IBMQ providers first")
            return
        
        assert(num_shots > 0)

        # Run the job on the least busy backend and store the results
        backend = least_busy(self.__provider.backends(filters=lambda b: b.configuration().n_qubits == 5 and
                                   not b.configuration().simulator and b.status().operational==True))

        job = backend.run(transpile(self.__qrn_circ, backend, optimization_level=3), 
                          memory=True, meas_level=2, shots=num_shots)
        job_monitor(job)
        result = job.result().get_memory()
            
        # Verify XOR-rule for obtained results
        random_gen = ""
        for measurement in result:
            if int(measurement[0]) ^ int(measurement[1]) != int(measurement[2]):
                print("Bad result found: " + measurement)
                continue

            # Append the measurement
            random_gen += measurement[0]
            
        if random_gen == "":
            self.__failed_results = True
            return

        self.__failed_results = False
        self.__random_gen = random_gen

In [50]:
qbg = QBitGen()
qbg.init_circ()
qbg.load_provider()
qbg.evaluate_circ(20)

ibmqfactory.load_account:WARNING:2022-04-21 16:39:37,700: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
Bad result found: 001
Bad result found: 100
Bad result found: 100
Bad result found: 100
Bad result found: 001


In [51]:
print(qbg.get_random())
print(len(qbg.get_random()))

100111100010000
15
